In [2]:
from pathlib import Path
import pandas as pd
import os
import csv
import numpy as np
import requests
import time
import io
import yfinance as yf
import warnings
# Initial imports
#%tensorflow_version 1.x
import tensorflow
import mibian
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lstm
import psutil
psutil.virtual_memory()

svmem(total=8589934592, available=2790621184, percent=67.5, used=5511163904, free=34963456, active=2655633408, inactive=1863249920, wired=2855530496)

In [3]:
##This is a place holder, the real dataframe we load will have the signals already implemented

df = pd.read_pickle('etf.pickle')
df

,optionkey,Symbol,ExpirationDate,AskPrice,AskSize,BidPrice,BidSize,LastPrice,PutCall,StrikePrice,Volume,openinterest,UnderlyingPrice,DataDate
0,MTUM2017-04-21c80.002017-03-22,MTUM,2017-04-21,3.70,5,1.50,5,0.00,call,80.00,0,0,82.34,2017-03-22
1,MTUM2017-04-21p80.002017-03-22,MTUM,2017-04-21,0.70,1,0.00,0,0.00,put,80.00,0,0,82.34,2017-03-22
2,MTUM2017-04-21c81.002017-03-22,MTUM,2017-04-21,2.90,5,0.65,5,0.00,call,81.00,0,0,82.34,2017-03-22
3,MTUM2017-04-21p81.002017-03-22,MTUM,2017-04-21,1.70,5,0.00,0,0.00,put,81.00,0,0,82.34,2017-03-22
4,MTUM2017-04-21c82.002017-03-22,MTUM,2017-04-21,2.15,5,0.00,0,0.00,call,82.00,0,0,82.34,2017-03-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262321,VOO2020-02-21P345.002019-12-30,VOO,2020-02-21,50.90,24,49.50,11,0,put,345.00,0,0,295.0400,2019-12-30
262322,VOO2020-02-21P350.002019-12-30,VOO,2020-02-21,55.80,25,54.50,22,0,put,350.00,0,0,295.0400,2019-12-30
262323,VOO2020-02-21P355.002019-12-30,VOO,2020-02-21,60.60,11,59.60,11,0,put,355.00,0,0,295.0400,2019-12-30
262324,VOO2020-02-21P360.002019-12-30,VOO,2020-02-21,65.80,22,64.30,139,0,put,360.00,0,0,295.0400,2019-12-30


In [8]:
# def test_reducer(df):
# # #     df = df.drop(colums# all unneded columns should be dropped)

In [9]:
# df = test_reducer(df)

In [10]:
### Change %out an absolute value, put signals out of the money are going to be negatives
### Or create multipliers of -1 for put targets

##

#atm = at the money
#ctm = close to the money
#otm = out of the money
#ftm = further out of the money
#xf = expire far
#xm = expiring medium's distance
#xs = expiring soon

atm = .07
ctm = .1
otm = .15
ftm = .2
xf = 28
xm = 19
xs = 7
def create_call_targets(df, atm, ctm, otm, xf, xm, xs):
    df['atm_xf'] = df['time delta'] < np.timedelta64(xf, 'D') & (df['% out'] > -.01) & (df['% out'] < atm)
    df['ctm_xf'] = df['time delta'] < np.timedelta64(xf, 'D') & (df['% out'] > atm) & (df['% out'] < ctm)
    df['otm_xf'] = df['time delta'] < np.timedelta64(xf, 'D') & (df['% out'] > ctm) & (df['% out'] < otm)
    df['ftm_xf'] = df['time delta'] < np.timedelta64(xf, 'D') & (df['% out'] > otm) & (df['% out'] < ftm)
    df['atm_xm'] = df['time delta'] < np.timedelta64(xm, 'D') & (df['% out'] > 0) & (df['% out'] < atm)
    df['ctm_xm'] = df['time delta'] < np.timedelta64(xm, 'D') & (df['% out'] > atm) & (df['% out'] < ctm)
    df['otm_xm'] = df['time delta'] < np.timedelta64(xm, 'D') & (df['% out'] > ctm) & (df['% out'] < otm)
    df['ftm_xm'] = df['time delta'] < np.timedelta64(xm, 'D') & (df['% out'] > otm) & (df['% out'] < ftm)
    df['atm_xs'] = df['time delta'] < np.timedelta64(xs, 'D') & (df['% out'] > 0) & (df['% out'] < atm)
    df['ctm_xs'] = df['time delta'] < np.timedelta64(xs, 'D') & (df['% out'] > atm) & (df['% out'] < ctm)
    df['otm_xs'] = df['time delta'] < np.timedelta64(xs, 'D') & (df['% out'] > ctm) & (df['% out'] < otm)
    return df

def create_put_targets(df, atm, ctm, otm, xf, xm, xs):
    df['atm_xf'] = df['time delta'] < np.timedelta64(xf, 'D') & (df['% out'] > .01) & (df['% out'] < (-1 * atm))
    df['ctm_xf'] = df['time delta'] < np.timedelta64(xf, 'D') & (df['% out'] > (-1 * atm)) & (df['% out'] < (-1 * ctm))
    df['otm_xf'] = df['time delta'] < np.timedelta64(xf, 'D') & (df['% out'] > (-1 * ctm)) & (df['% out'] < (-1 * otm))
    df['ftm_xf'] = df['time delta'] < np.timedelta64(xf, 'D') & (df['% out'] > (-1 * otm)) & (df['% out'] < (-1 * ftm))
    df['atm_xm'] = df['time delta'] < np.timedelta64(xm, 'D') & (df['% out'] >  0) & (df['% out'] < (-1 * atm))
    df['ctm_xm'] = df['time delta'] < np.timedelta64(xm, 'D') & (df['% out'] > (-1 * atm)) & (df['% out'] < (-1 * ctm))
    df['otm_xm'] = df['time delta'] < np.timedelta64(xm, 'D') & (df['% out'] > (-1 * ctm)) & (df['% out'] < (-1 * otm))
    df['ftm_xm'] = df['time delta'] < np.timedelta64(xm, 'D') & (df['% out'] > (-1 * otm)) & (df['% out'] < (-1 * ftm))
    df['atm_xs'] = df['time delta'] < np.timedelta64(xs, 'D') & (df['% out'] > (-1 * 0) )& (df['% out'] <(-1 *  atm))
    df['ctm_xs'] = df['time delta'] < np.timedelta64(xs, 'D') & (df['% out'] > (-1 * atm)) & (df['% out'] < (-1 * ctm))
    df['otm_xs'] = df['time delta'] < np.timedelta64(xs, 'D') & (df['% out'] > (-1 * ctm)) & (df['% out'] < (-1 * otm))
    return df

In [11]:
#
def prepare_test(df):
    ##using map exp function already built
    df = map_exp(df)
    ## select options or puts df
    df = select_df(df, symbol, option)
    #create spread for potential spread strategy
    df = create_spread(df)

In [12]:
###Start appending values and creating csvs
### Do this for every variation


df['bl_1'] = np.random.randint(low = 0, high = 2, size = len(df))
df['LSTM'] = np.random.randint(low = 0, high = 2, size = len(df))


#add every test to a master frame?


def testing(df):
    
    bl_1_LTSM = df.loc[(df['bl_1']== 1) & (df['LSTM'] == 1)]
    bl_1_LTSM['log'] = bl_1_LTSM['AskPrice'].astype(float) - bl_1_LTSM['BidPrice'].astype(float)
    bl_1_LTSM_total = bl_1_LTSM['log'].sum()
#     bl_1_LTSM = short_close['log'].sum()
    bl_1_LTSM_log = bl_1_LTSM['log']
#     bl_1_LTSM_log_df = pd.DataFrame(bl_1_LTSM, columns = ['log'])
    bl_1_LTSM['log'].to_csv('bl_1_LTSM.csv')
    df['bl_1_LTSM_log'] = bl_1_LTSM['log']
    return df
    
    


In [13]:
df

,optionkey,Symbol,ExpirationDate,AskPrice,AskSize,BidPrice,BidSize,LastPrice,PutCall,StrikePrice,Volume,openinterest,UnderlyingPrice,DataDate,bl_1,LSTM
0,MTUM2017-04-21c80.002017-03-22,MTUM,2017-04-21,3.70,5,1.50,5,0.00,call,80.00,0,0,82.34,2017-03-22,1,0
1,MTUM2017-04-21p80.002017-03-22,MTUM,2017-04-21,0.70,1,0.00,0,0.00,put,80.00,0,0,82.34,2017-03-22,1,0
2,MTUM2017-04-21c81.002017-03-22,MTUM,2017-04-21,2.90,5,0.65,5,0.00,call,81.00,0,0,82.34,2017-03-22,0,0
3,MTUM2017-04-21p81.002017-03-22,MTUM,2017-04-21,1.70,5,0.00,0,0.00,put,81.00,0,0,82.34,2017-03-22,0,0
4,MTUM2017-04-21c82.002017-03-22,MTUM,2017-04-21,2.15,5,0.00,0,0.00,call,82.00,0,0,82.34,2017-03-22,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262321,VOO2020-02-21P345.002019-12-30,VOO,2020-02-21,50.90,24,49.50,11,0,put,345.00,0,0,295.0400,2019-12-30,0,0
262322,VOO2020-02-21P350.002019-12-30,VOO,2020-02-21,55.80,25,54.50,22,0,put,350.00,0,0,295.0400,2019-12-30,0,1
262323,VOO2020-02-21P355.002019-12-30,VOO,2020-02-21,60.60,11,59.60,11,0,put,355.00,0,0,295.0400,2019-12-30,1,0
262324,VOO2020-02-21P360.002019-12-30,VOO,2020-02-21,65.80,22,64.30,139,0,put,360.00,0,0,295.0400,2019-12-30,0,1


In [14]:
trial = testing(df)

<ipython-input-12-4360ee087e6b>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bl_1_LTSM['log'] = bl_1_LTSM['AskPrice'].astype(float) - bl_1_LTSM['BidPrice'].astype(float)


ValueError: cannot reindex from a duplicate axis

In [ ]:
trial

In [ ]:
#Create signal columns that will enter us in to a strategy


###To be added, when br_1 is true and RF is -1, add 1
###replicate for bl and LTSM



def basic_signals(df):
    #define emoty series for br and bl
    br_1 = []
    br_2 = []
    bl_1 = []
    bl_2 = []
    for index, row in df.iterrows():
        # Put signals and bollinger_signal -1 determines bearish signal
        if (row['Put signals'] == True & row['bollinger_signal'] == -1):
            br_1.append(True)
        else:
            br_1.append(False)
        # Put signals and relative volatility 0 determines bearish signal
        if (row['Put signals'] == True & row['rvol signal'] == 0):
            br_2.append(True)
        else:
            br_2.append(False)
        #high volatility + a put/call signal could identify stock movement
        if (row['rvol signal'] == 1 & row['Call signals'] == True):
            bl_1.append(True)
        else:
            bl_1.append(False)
        #Put/call signal and Option rvol signals a bullish movement
        if (row['Call signals'] == True and row['Option rVol'] > 1.4):
            bl_2.append(True)
        else:
            bl_2.append(False)
    #append series columns to dataframe
    df['br_1'] = br_1
    df['br_2'] = br_2
    df['bl_1'] = bl_1
    df['bl_2'] = bl_2
    return df